
GTFS Wiki: http://wiki.corp.nextbus.com/nextbuswiki/GtfsChecks (need to be on NexBus VPN) 

GTFS Goolge Guideline: https://developers.google.com/transit/gtfs/reference/

# <span style="color:red">Import the needed packages</span>

In [1]:
import pandas as pd
import numpy as np
#import pixiedust as pix # will look into the capabilites of this library

### <span style="color:green">Add the location of your GTFS data</span>

In [36]:
# for windows
# filepath = r"\\cts-con-fld-001\users$\207329\My Documents\qa\vacaville\vacavillecitycoach-ca-us_20181015"

# for linux
filepath = r"/mnt/shared/qa/bowvalley"

# filepath2 = r"/mnt/shared/qa/seattle-sc/Seattle-sc-ALL_20181126_google_daily_transit"



'''Some functions to make life eaiser
Note if you are using Windows, you'll need to modify the "/" '''




# load file and show top 5 records
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    return pd.read_csv(f)

# load GTFS files
agency = load('agency')
trips = load('trips')
stops = load('stops')
stop_times = load('stop_times')
shapes = load('shapes')
routes = load('routes')
calendar = load('calendar')
calendar_dates = load('calendar_dates')

import csv
# short save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '//' + "{}".format(fName), sep =',', index=False)#, quoting=csv.QUOTE_NONNUMERIC)
    return print("Saved {} to {}".format(fName,filepath))

# short find null
def null(fName):
    return df[(fName).isnull()]

# drop field (field name, file name)
def drop(fName='timepoint',fileName='stop_times'):
    df.drop(fName, 1, inplace=True)
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Dropped '{}' field.\n\
Saved '{}_FIXED.txt' to file folder.".format(fName,fileName))

# drop dups in blocks.txt for CNB 
def dups():
    #df[df.block_id.duplicated()]
    df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
    return df1.info()
    
# sort sequence for stop_times.txt file
def sort():
    df.sort_values(['trip_id', 'stop_sequence'])
    return df.head(15)

# shape less than certain amount of points for shapes
def lessthan(x):
    df1 = df.groupby(['shape_id']).count()
    return df1[(df1.shape_pt_lat < x)]

# function to move copy stop_code to stop_id in the stops.txt file
def copyField(newField='stop_code',origField='stop_id',fileName='stops'):
    df[newField] = df[origField]
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Copied the '{}' field to the '{}' field.\n\
Saved '{}_FIXED.txt' to file folder.".format(origField,newField,fileName))




### work for CNB shape issues
# def shapeFix(indx1,indx2,shapeName):
#     x1 = int(indx1)
#     x2 = int(indx2)
#     global df
#     df = df.drop(df.index[x1:x2]) # drop double shape pnts 
#     df = df[(df.shape_id == '{}'.format(shapeName))] # create new df of just shape pnts
#     df['shape_pt_sequence'] = range(1,1 + len(df)) # re-sequence shape pnts
#     df_cut = df # give unique name to subset of shape pnts

#     # load orig data, drop all of courthou1, append 
#     load('shapes') # load orig df
#     df = df[df.shape_id != '{}'.format(shapeName)] # drop all cases of shape 
#     global df_all
#     df_all = df.append(df_cut,ignore_index=True) # create new df and append df_cut
#     return  print('use df_all')



## Random Work-flows

In [ ]:
## fill na with number, query and output file  
df['block_id'].fillna(999999, inplace=True)
df1 = df[(df.block_id == 999999)]
df['block_id'].unique()
save(df1,'trips_missingBlocks.txt')

# compare two files
# set the field to a list
oldTrips = df2['trip_id'].unique().tolist()
newTrips = df['trip_id'].unique().tolist()

oldBlocks = df2['block_id'].unique().tolist()
newBlocks = df['block_id'].unique().tolist()

# use set and difference to compare the two list
set(oldTrips).difference(newTrips)

set(newBlocks).difference(oldBlocks)

# can alsow use this logic... but the above works just fine
if (40803723 in newTrips):
    print ("Exist")
else:
    print("Nope")

##### seattle-sc 'like' function 
df=df.loc[df['route_id'].isin([100340,102638])]
df
save(df,'trips')
df['service_id'].unique()
df2=df2.loc[df2['route_id'].isin([100340,102638])]

#### create new column based on conditions of other columns
df['timeCheck'] = np.where(df['arrival_time']==df['departure_time'],'yes','no')
df1 = df[(df.timeCheck=='no')]
df['timeCheck'].unique()

# syntax for leading zeros (4 for example)
df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))


# <span style="color:red">agency.txt</span>

In [37]:
agency = load('agency')

,agency_name,agency_url,agency_timezone,agency_lang
0,Roam Transit,http://www.roamtransit.com,America/Edmonton,en


# <span style="color:red">routes.txt</span>

### Required Fields: 
1. route_id- These are the routes that are shown to the public. Route ID's cannot have leading zeros, asterisks, underscore, dashes and perhaps other characters. 
2. route_type- numberic value with specific meanings:
    * 0 Tram, Streetcar, Light rail
    * 1 Subway, Metro
    * 2 Rail
    * 3 Bus
    * 4 Ferry
    * 5 Cable Car

In [58]:
# just use the funtion 'load()' to load the .txt file and see the first 5 lines ex: loadfile('routes')
routes = load('routes')
cols = ['route_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_type', 'route_color', 'route_url','route_text_color']
routes = routes[cols]
routes.head()

,route_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_url,route_text_color
0,59,Training,NaN,NaN,3,F9FF40,NaN,000000
1,22,8S,Route 8S - Scenic Lake Louise Bound,NaN,3,859969,https://roamtransit.com/wp-content/uploads/201...,000000
2,17,6,Route 6 - Lake Minnewanka Banff Bound,This route provides service to Lake Minnewanka...,3,0077C8,https://roamtransit.com/wp-content/uploads/201...,FFFFFF
3,33,4,Route 4 - Cave and Basin Cave Bound,Roam Route 4 provides service from the length ...,3,FFB500,https://roamtransit.com/wp-content/uploads/201...,000000
4,27,2,Route 2 - Tunnel Mountain Banff Spring Hotel B...,Roam Route 2 provides service from the top of ...,3,00AFD7,https://roamtransit.com/wp-content/uploads/201...,000000


# <span style="color:red">trips.txt</span>

### Required Fields:
1. route_id- Check to see if there are trips for every route. 
2. service_id- Check to see if the value of this field comes from the calendar.txt file. (below)
3. trip_id
4. direction_id- Can either be 1 or 0. Can omit if all routes are in one direction. 
5. block_id- GTFS list this as optional, NextBus requires it. Make sure there are no missig values. 
6. shape_id- GTFS list this as optional, NextBus requires it. This field links with the shapes.txt file.(below)
   


In [79]:
trips = load('trips')
trips.shape_id.astype(np.int64) # only if no nan


ValueError: Cannot convert non-finite values (NA or inf) to integer

# <span style="color:red">stops.txt</span>

### Required Fields:
1. stop_id 
2. stop_lat- Make sure none are zero, or empty.
3. stop_lon- Make sure none are zero, or empty.
    - make sure lat and long are at least five decimal places (most do six) 

In [61]:
# stops.columns = [['stop_id', 'stop_name', 'stop_desc','stop_lat', 'stop_lon']]
stops = load('stops')
cols = ['stop_id', 'stop_name', 'stop_desc', 'stop_lat', 'stop_lon' ]
stops = stops[cols]
stops.head()


,stop_id,stop_name,stop_desc,stop_lat,stop_lon
0,131,Two Jack Lakeside Banff Bound,NaN,51.222501,-115.500851
1,132,Two Jack Main Banff Bound,NaN,51.226482,-115.504577
2,133,Lake Minnewanka,NaN,51.247584,-115.499923
3,134,Minnewanka Lookout Lake Bound,NaN,51.235920,-115.488315
4,135,Elk Street Transit Hub,NaN,51.179164,-115.570642


# <span style="color:red">stop_times.txt</span>

### Required Fields:
1. trip_id
2. arrival_time- Should be blank for non-scheduled stops
3. depature_times- Should be blank for non-scheduled stops
4. stop_id
5. stop_sequence- Some agencies will sort this field incorrectly, putting all the "1s" first.
    - note* watch out for the timepoints field, if there is a 1, then it is an acutal stop. If there are not enough of 1s, this may cause problems. It's okay to delete this field. 

In [67]:
stop_times = load('stop_times') 
cols = ['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence']
stop_times =  stop_times[cols]
stop_times.head()


,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,0,09:30:00,09:30:00,222,0
1,0,09:32:00,09:32:00,251,1
2,0,10:00:00,10:00:00,188,2
3,0,10:08:00,10:08:00,234,3
4,0,10:10:00,10:10:00,184,4


# <span style="color:red">shapes.txt</span>

### Required Fields:
1. shape_id
2. shape_pnt_lat- at least five decimal places
3. shape_pnt_lon- at least five deciaml places

In [68]:
shaps = load('shapes')
shapes.head()


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,22,51.17963,-115.56994,0
1,22,51.18083,-115.56872,1
2,22,51.18139,-115.57010,2
3,22,51.18058,-115.57084,3
4,22,51.17987,-115.57162,4


# <span style="color:red">calendar.txt</span>

### Required:
Run the calendar script if more than one service record is defined for the day of the week.
* If there is more than one service defined for the week, then run the calendar.py script.


In [69]:
calendar = load('calendar')
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,59,1,1,1,1,1,1,1,20190610,20190915
1,22,1,1,1,1,1,1,1,20190621,20190915
2,17,1,1,1,1,1,1,1,20190517,20190915
3,33,1,1,1,1,1,1,1,20190517,20190915
4,27,1,1,1,1,1,1,1,20190517,20190915
5,55,1,1,1,1,1,1,1,20190602,20190915
6,29,1,1,1,1,1,1,1,20190621,20190915
7,60,1,1,1,1,1,1,1,20190517,20190915
8,46,1,1,1,1,1,1,1,20190602,20190915
9,42,1,1,1,1,1,0,0,20190517,20190705


# <span style="color:red">calendar_dates.txt</span>

### Required:
This is an optional file, but is used to mark special holidays. Make sure the dates make sense.

In [70]:
calendar_dates = load('calendar_dates')
calendar_dates


,service_id,date,exception_type
0,46,20190902,2
1,46,20190805,2
2,46,20190701,2
3,46,20190520,2
4,42,20190902,2
5,42,20190805,2
6,42,20190701,2
7,42,20190520,2
